In [1]:
import heapq
import geopy.distance
import math
import csv
import Utils
import time
  
PUBLIC_TRANSPORT_SPEED_KM_H = 10

class Graph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for start, end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon in self.edges:
            if start in self.graph_dict:
                self.graph_dict[start].append((end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon))
            else:
                self.graph_dict[start] = [(end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon)]
            if end not in self.graph_dict:
                self.graph_dict[end] = []

class LineGraph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for _, end, line, _, arrival_time, _, _, _, _, _ in self.edges:
            if end in self.graph_dict:
                self.graph_dict[end].add(line)
            else:
                self.graph_dict[end] = {line}

class LineAndTimeGraph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for _, end, line, _, arrival_time, _, _, _, _, _ in self.edges:
            if (end, line) in self.graph_dict:
                self.graph_dict[(end, line)].add(arrival_time)
            else:
                self.graph_dict[(end, line)] = {arrival_time}
            
def edges(fileName):
    edges = []
    with open(fileName, 'r', encoding='utf8') as file:
        reader = csv.reader(file)
        header = next(reader)
        
        for row in reader:
            line = row[2]
            start_time = Utils.minutes_from_midnight(row[3])
            arrival_time = Utils.minutes_from_midnight(row[4]) 
            weight = arrival_time - start_time
            
            if weight < 0:
                weight += 24*60
            
            start = row[5]
            end = row[6]
            
            start_stop_lat = row[7]
            start_stop_lon = row[8]
            end_stop_lat = row[9]
            end_stop_lon = row[10]
            
            edges.append((start, end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon))
    return edges


def distance_from_coords(coords1, coords2):
    return geopy.distance.geodesic(coords1, coords2).km

def time_from_distance(distance):
    return distance/PUBLIC_TRANSPORT_SPEED_KM_H * 60
    
def time_from_coords(coords1, coords2):
    return time_from_distance(distance_from_coords(coords1, coords2))


In [2]:
edges = edges('connection_graph_fixed.csv')
gg = Graph(edges)
lg = LineGraph(edges)
lng = LineAndTimeGraph(edges)

In [3]:
def get_best_direct_line(graph, goal_lines, line_n_time_graph, start_time, optimistic_arrival_time, goal, start):
    fastest_direct_time = 25*60
    best_line = None
    lines_visited = []
    for _, line, _, _, _, _, _, _, _ in graph[start]:
        if line in lines_visited:
            continue
        else:
            lines_visited.append(line)
            
        if line in goal_lines:
            best_line_time = 25*60
            for arrival_time in line_n_time_graph[goal, line]:
                if arrival_time >= optimistic_arrival_time:
                    if arrival_time - start_time < best_line_time:
                        best_line_time = arrival_time - start_time
                elif arrival_time + 24*60 - start_time < best_line_time:
                    best_line_time = arrival_time + 24*60 - start_time
            if best_line_time < fastest_direct_time:
                fastest_direct_time = best_line_time
                best_line = line
    return best_line, fastest_direct_time

def get_first_direct_line(graph, goal_lines, start_time, start):
    min_direct_wait_time = 25*60
    best_line = None
    curr_line = None
    for _, line, line_start_time, _, _, _, _, _, _ in graph[start]:
        if curr_line != line:
            min_wait_time = 25*60
            curr_line = line
            
        if line in goal_lines:
            if line_start_time >= start_time:
                if line_start_time - start_time < min_wait_time:
                    min_wait_time =  line_start_time - start_time
            elif line_start_time + 24*60 - start_time < min_wait_time:
                min_wait_time =  line_start_time + 24*60 - start_time
        
            if min_wait_time < min_direct_wait_time:
                min_direct_wait_time = min_wait_time
                best_line = line          

    return best_line, min_direct_wait_time

In [4]:
def distance_from_coords(coords1, coords2):
    return geopy.distance.geodesic(coords1, coords2).km

def distance_from_coords_manhatann(coords1, coords2):
        coords1_lat, coords1_lon = coords1
        coords2_lat, coords2_lon = coords2
        return geopy.distance.geodesic((coords1_lat, coords1_lon), (coords2_lat, coords1_lon)).km + geopy.distance.geodesic((coords1_lat, coords1_lon), (coords1_lat, coords2_lon)).km

def time_from_distance(distance):
    return distance/PUBLIC_TRANSPORT_SPEED_KM_H * 60
    
def time_from_coords(coords1, coords2):
    return time_from_distance(distance_from_coords(coords1, coords2))


In [5]:
LINE_CHANGE_COST = 20

def astar_p(graph_dict, lines_dict, start, goal, trip_start_time, coords_heuristic_fn, starting_line):
    goal_info = graph_dict[goal][0]
    goal_coords = (goal_info[5], goal_info[6])
    
    start_info = graph_dict[start][0]
    current_coords = (start_info[5], start_info[6])
    
    direct_line = None
    counter = 0
    
    pq = [(0, start)]
    prev_nodes = {start: None}
    distances = {node: float('inf') for node in graph_dict}
    distances[start] = 0
    
    fScore = {node: float('inf') for node in graph_dict}
    fScore[start] = 0
    
    trip_start_time = Utils.minutes_from_midnight(trip_start_time)

    isFirstStop = True
    
    while pq:
        _, current = heapq.heappop(pq)
        curr_cost = distances[current]
        curr_f_score = fScore[current]
        curr_arrival_time = trip_start_time + curr_cost
        base_days_en_route = 0
        if current == goal:
            break
        
        if curr_arrival_time > 24*60:
            base_days_en_route = int(curr_arrival_time/(24*60))
            curr_arrival_time = curr_arrival_time%(24*60)
            
        prev_line = ''
        if prev_nodes[current]:
            prev_line = prev_nodes[current][1]
            
        for neighbor, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, _, _ in graph_dict[current]:
                
            # if there is a direct connection, but this aint it
            if direct_line:
                if line != direct_line:
                    continue
            elif line in lines_dict[goal]:       
                optimistic_arrival_time = curr_arrival_time + math.floor(coords_heuristic_fn(current_coords, goal_coords))
                # direct_line, best_time = get_best_direct_line(graph_dict, lines_dict[goal], line_n_time_graph, curr_arrival_time, optimistic_arrival_time, goal, current)
                # if best_time > (optimistic_arrival_time - curr_arrival_time) * 3:
                #     direct_line = None
                direct_line, best_time = get_first_direct_line(graph_dict, lines_dict[goal], curr_arrival_time, current)
                if best_time > LINE_CHANGE_COST:
                    direct_line = None
                if line != direct_line:
                    continue
                
            if prev_line == line or prev_line == '': line_change_penalty = 0
            else: line_change_penalty = LINE_CHANGE_COST 
            
            if isFirstStop and starting_line and starting_line != line:
                line_change_penalty += LINE_CHANGE_COST
            
            counter += 1
            days_en_route = base_days_en_route
            current_coords = (start_stop_lat, start_stop_lon)

            if curr_arrival_time <= start_time:
                # current travel time + time spent waiting for the bus/tram + time spent en route 
                new_cost = curr_cost + (start_time - (curr_arrival_time)) + weight
            else:
                # current travel time + (time till midnight + start_time) + time spent en route
                new_cost = curr_cost + (24*60 - (curr_arrival_time) + start_time) + weight
                days_en_route += 1
            
            if curr_f_score + new_cost + line_change_penalty < fScore[neighbor]:
                distances[neighbor] = new_cost
                fScore[neighbor] = curr_f_score + new_cost + line_change_penalty
                priority = curr_f_score + new_cost + math.floor(coords_heuristic_fn(current_coords, goal_coords)) + line_change_penalty

                heapq.heappush(pq, (priority, neighbor))
                prev_nodes[neighbor] = (current, line, start_time + days_en_route*24*60, arrival_time + days_en_route*24*60)
        isFirstStop = False
    # print(f'COUNTER: {counter}')
    return distances, prev_nodes

In [6]:
def astar_t(graph_dict, start, goal, trip_start_time, coords_heuristic_fn):
    goal_info = graph_dict[goal][0]
    coords2 = (goal_info[5], goal_info[6])
    counter = 0
    pq = [(0, start)]
    prev_nodes = {start: None}
    
    distances = {node: float('inf') for node in graph_dict}
    distances[start] = 0
    
    fScore = {node: float('inf') for node in graph_dict}
    fScore[start] = 0
    
    trip_start_time = Utils.minutes_from_midnight(trip_start_time)
    while pq:
        curr_f_score, current = heapq.heappop(pq)
        curr_cost = distances[current]
        curr_arrival_time = trip_start_time + curr_cost
        base_days_en_route = 0
        if current == goal:
            break
        
        if curr_f_score > fScore[current]:
            continue
        
        if curr_arrival_time > 24*60:
            base_days_en_route = int(curr_arrival_time/(24*60))
            curr_arrival_time = curr_arrival_time%(24*60)
            
        prev_line = ''
        if prev_nodes[current]:
            prev_line = prev_nodes[current][1]
            
        for neighbor, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon in graph_dict[current]:
            counter += 1
            days_en_route = base_days_en_route
            
            coords1 = (start_stop_lat, start_stop_lon)

            if curr_arrival_time <= start_time:
                # current travel time + time spent waiting for the bus/tram + time spent en route 
                new_cost = curr_cost + (start_time - (curr_arrival_time)) + weight
            else:
                # current travel time + (time till midnight + start_time) + time spent en route
                new_cost = curr_cost + (24*60 - (curr_arrival_time) + start_time) + weight
                days_en_route += 1
            
            if new_cost < distances[neighbor]:
                distances[neighbor] = new_cost
                priority = new_cost + math.floor(coords_heuristic_fn(coords1, coords2))
                fScore[neighbor] = priority
                heapq.heappush(pq, (priority, neighbor))
                prev_nodes[neighbor] = (current, line, start_time + days_en_route*24*60, arrival_time + days_en_route*24*60)

    # print(f'COUNTER: {counter}')
    return distances, prev_nodes


In [7]:
def shortest_path_t(graph_dict, start, goal, trip_start_time, coords_heuristic_fn):
    calc_start_time = time.time()
    distances, prev_nodes = astar_t(graph_dict, start, goal, trip_start_time, coords_heuristic_fn)
    calc_end_time = time.time()
    print(f'time_taken: {round(calc_end_time - calc_start_time, 3)}s')
    path = []
    curr_node = goal
    line = ''
    trip_length = distances[goal]
    start_time = 0
    arrival_time = Utils.minutes_from_midnight(trip_start_time) + trip_length
    while True:
        path.append((curr_node, line, start_time, arrival_time))
        if(prev_nodes[curr_node]):
            curr_node, line, start_time, arrival_time = prev_nodes[curr_node]
        else:
            break
    path.reverse()
    Utils.print_solution(trip_start_time, trip_length, path)
    return Utils.minutes_from_midnight(trip_start_time) + trip_length
   
def shortest_path_p(graph_dict, lines_dict, start, goal, trip_start_time, coords_heuristic_fn, starting_line):
    calc_start_time = time.time()
    distances, prev_nodes = astar_p(graph_dict, lines_dict, start, goal, trip_start_time, coords_heuristic_fn, starting_line)
    calc_end_time = time.time()
    print(f'time_taken: {round(calc_end_time - calc_start_time, 3)}s')
    path = []
    curr_node = goal
    line = ''
    trip_length = distances[goal]
    start_time = 0
    arrival_time = Utils.minutes_from_midnight(trip_start_time) + trip_length
    while True:
        path.append((curr_node, line, start_time, arrival_time))
        if(prev_nodes[curr_node]):
            curr_node, line, start_time, arrival_time = prev_nodes[curr_node]
        else:
            break
    path.reverse()
    Utils.print_solution(trip_start_time, trip_length, path)
    last_used_line = path[:-2][1]
    return Utils.minutes_from_midnight(trip_start_time) + trip_length, last_used_line

def printTabu_t(stops, startTime, grapth_dict):
    start_time = startTime
    for i in range(stops.__len__()):
        start_time = Utils.to_time(shortest_path_t(grapth_dict, stops[i], stops[(i+1)%stops.__len__()], start_time, distance_from_coords))
                
def printTabu_p(stops, startTime, grapth_dict, lines_dict, startLine):
    start_line = startLine
    start_time = startTime
    for i in range(stops.__len__()):
        start_time, start_line = shortest_path_p(grapth_dict, lines_dict, stops[i], stops[(i+1)%stops.__len__()], start_time, distance_from_coords, start_line)
        start_time = Utils.to_time(start_time)

In [17]:
import random
import math

def Tabu_t(stops, trip_start_time, graph_dict):
    calc_start_time = time.time()
    n_stops = len(stops)
    max_iterations = math.ceil(1.1*(n_stops**2))
    turns_improved = 0
    improve_thresh=2*math.floor(math.sqrt(max_iterations))
    tabu_list = []
    tabu_tenure = n_stops
    # tabu_tenure = 100000000

    distances = [[astar_t(graph_dict, city1, city2, trip_start_time, distance_from_coords)[0][city2] for city1 in stops] for city2 in stops]

    total=0
    for i in range(n_stops):
            for j in range(n_stops):
                total += distances[i][j]

    aspiration_criteria = (total/(n_stops**2))*2.2
    # aspiration_criteria = -1

    current_solution = list(range(n_stops))
    best_solution = current_solution[:]
    best_solution_cost = sum([distances[current_solution[i]][current_solution[(i+1)%n_stops]] for i in range(n_stops)])

    for iteration in range(max_iterations):
        if turns_improved>improve_thresh:
            break
        best_neighbor = None
        best_neighbor_cost = float('inf')
        tabu_candidate = (0,0)
        for i in range(n_stops):
            for j in range(i+1, n_stops):
                neighbor = current_solution[:]
                if i > 0:
                    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbor_cost = sum([distances[neighbor[i]][neighbor[(i+1)%n_stops]] for i in range(n_stops)])
                if (i,j) not in tabu_list or neighbor_cost < aspiration_criteria:
                    if neighbor_cost < best_neighbor_cost:
                        best_neighbor = neighbor[:]
                        best_neighbor_cost = neighbor_cost
                        tabu_candidate = (i,j)
        if best_neighbor is not None:
            current_solution = best_neighbor[:]
            tabu_list.append(tabu_candidate)
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)
            if best_neighbor_cost < best_solution_cost:
                best_solution = best_neighbor[:]
                best_solution_cost = best_neighbor_cost
                turns_improved=0
            else:
                turns_improved=turns_improved+1

        # print("Iteration {}: Best solution cost = {}".format(iteration, best_solution_cost))



    stops_sorted = [stops[i] for i in best_solution]
    print(f"Best solution: {stops_sorted}")    
    printTabu_t(stops_sorted, trip_start_time, gg.graph_dict)
    
    calc_end_time = time.time()
    # print(f"time: {calc_end_time - calc_start_time}")
    # print("Best solution cost: {}".format(best_solution_cost))
    
def Tabu_p(stops, trip_start_time, graph_dict, lines_dict):
    calc_start_time = time.time()
    n_stops = len(stops)
    max_iterations = math.ceil(1.1*(n_stops**2))
    turns_improved = 0
    improve_thresh= math.floor(math.sqrt(max_iterations))
    tabu_list = []
    tabu_tenure = n_stops
    # tabu_tenure = 100000000

    distances = [[astar_t(graph_dict, stop1, stop2, trip_start_time, distance_from_coords)[0][stop2] for stop1 in stops] for stop2 in stops]

    total=0
    for i in range(n_stops):
            for j in range(n_stops):
                total += distances[i][j]

    aspiration_criteria = (total/(n_stops**2))*2.2
    # aspiration_criteria = -1

    current_solution = list(range(n_stops))
    best_solution = current_solution[:]
    best_solution_cost = sum([distances[current_solution[i]][current_solution[(i+1)%n_stops]] for i in range(n_stops)])

    for iteration in range(max_iterations):
        if turns_improved>improve_thresh:
            break
        best_neighbor = None
        best_neighbor_cost = float('inf')
        tabu_candidate = (0,0)
        for i in range(n_stops):
            for j in range(i+1, n_stops):
                neighbor = current_solution[:]
                if i > 0:
                    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbor_cost = sum([distances[neighbor[i]][neighbor[(i+1)%n_stops]] for i in range(n_stops)])
                if (i,j) not in tabu_list or neighbor_cost < aspiration_criteria:
                    if neighbor_cost < best_neighbor_cost:
                        best_neighbor = neighbor[:]
                        best_neighbor_cost = neighbor_cost
                        tabu_candidate = (i,j)
        if best_neighbor is not None:
            current_solution = best_neighbor[:]
            tabu_list.append(tabu_candidate)
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)
            if best_neighbor_cost < best_solution_cost:
                best_solution = best_neighbor[:]
                best_solution_cost = best_neighbor_cost
                turns_improved=0
            else:
                turns_improved=turns_improved+1

        # print("Iteration {}: Best solution cost = {}".format(iteration, best_solution_cost))
        

    stops_sorted = [stops[i] for i in best_solution]
    print(f"Best solution: {stops_sorted}")
    printTabu_p(stops_sorted, trip_start_time, gg.graph_dict, lines_dict, None)
    
    calc_end_time = time.time()
    print(f"time: {calc_end_time - calc_start_time}")
    # print("Best solution cost: {}".format(best_solution_cost))
    

In [18]:
stops1 = ['Kwiska', 'PL. GRUNWALDZKI', 'Ogród Botaniczny', 'DWORZEC GŁÓWNY', 'GIEŁDOWA (Centrum Hurtu)']
stops2 = ['pl. Bema', 'Łozina - Wrocławska (na wys. nr 18)', 'Kwiska', 'DWORZEC GŁÓWNY', 'GIEŁDOWA (Centrum Hurtu)']
stops3 = ['pl. Bema', 'PL. GRUNWALDZKI', 'Kwiska']
stops4 = ['DWORZEC GŁÓWNY', 'Łozina - Wrocławska (na wys. nr 18)', 'GIEŁDOWA (Centrum Hurtu)']

stops5 = ['PL. GRUNWALDZKI', 'Kwiska', 'GALERIA DOMINIKAŃSKA']

trip_start_time = "12:00:00"
Tabu_t(stops1, trip_start_time, gg.graph_dict)
Tabu_t(stops2, trip_start_time, gg.graph_dict)
Tabu_t(stops3, trip_start_time, gg.graph_dict)
Tabu_t(stops4, trip_start_time, gg.graph_dict)
Tabu_t(stops5, trip_start_time, gg.graph_dict)



Best solution: ['Kwiska', 'Ogród Botaniczny', 'PL. GRUNWALDZKI', 'DWORZEC GŁÓWNY', 'GIEŁDOWA (Centrum Hurtu)']
Best solution: ['pl. Bema', 'Łozina - Wrocławska (na wys. nr 18)', 'GIEŁDOWA (Centrum Hurtu)', 'Kwiska', 'DWORZEC GŁÓWNY']
Best solution: ['pl. Bema', 'Kwiska', 'PL. GRUNWALDZKI']
Best solution: ['DWORZEC GŁÓWNY', 'Łozina - Wrocławska (na wys. nr 18)', 'GIEŁDOWA (Centrum Hurtu)']
Best solution: ['PL. GRUNWALDZKI', 'Kwiska', 'GALERIA DOMINIKAŃSKA']


In [15]:
Tabu_p(stops1, trip_start_time, gg.graph_dict, lg.graph_dict)
Tabu_p(stops2, trip_start_time, gg.graph_dict, lg.graph_dict)
Tabu_p(stops3, trip_start_time, gg.graph_dict, lg.graph_dict)
Tabu_p(stops4, trip_start_time, gg.graph_dict, lg.graph_dict)
Tabu_p(stops5, trip_start_time, gg.graph_dict, lg.graph_dict)

Best solution: ['Kwiska', 'Ogród Botaniczny', 'PL. GRUNWALDZKI', 'DWORZEC GŁÓWNY', 'GIEŁDOWA (Centrum Hurtu)']
time_taken: 0.205s
time: 08:55:00
start time: 12:00:00
in: Kwiska [12:00:00]| line: 3
out: Małopanewska [12:01:00]| line: 3
in: Małopanewska [12:07:00]| line: 10
out: Ogród Botaniczny [20:55:00]| line: 10
-
-
-
time_taken: 0.019s
time: 00:16:00
start time: 20:55:00
in: Ogród Botaniczny [21:06:00]| line: N
out: Katedra [21:07:00]| line: N
in: Katedra [21:07:00]| line: 10
out: PL. GRUNWALDZKI [21:11:00]| line: 10
-
-
-
time_taken: 0.139s
time: 02:23:00
start time: 21:11:00
in: PL. GRUNWALDZKI [21:12:00]| line: 10
out: DWORZEC GŁÓWNY [23:34:00]| line: 10
-
-
-
time_taken: 0.111s
time: 04:09:00
start time: 23:34:00
in: DWORZEC GŁÓWNY [23:34:00]| line: 3
out: DWORZEC AUTOBUSOWY [23:36:00]| line: 3
in: DWORZEC AUTOBUSOWY [00:21:00]| line: 257
out: GIEŁDOWA (Centrum Hurtu) [03:43:00]| line: 257
-
-
-
time_taken: 0.017s
time: 00:21:00
start time: 27:43:00
in: GIEŁDOWA (Centrum Hurtu) 